# Merge all auctions into one

Combines several datasets and do some simple analyses.  


### User variables

In [1]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
AuctionDates = ['2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
                '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12']  
#AuctionDates = ['2019-11']

### Import modules 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
plt.style.use('ggplot')

### Load all data

Auctions are saved monthly in a .pkl file

In [3]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    fn = '../data/rdw-data-{:s}.pkl'.format(
    # fn = '../../../python-nb/data/rdw-data-opbod-{:s}.pkl'.format(
        AuctionDate
    )
    print(fn)
    df = pd.read_pickle(fn)
    data[AuctionDate] = df

print('\nlast rows of last file')
data[AuctionDates[-1]].tail()

load data
../data/rdw-data-2017-03.pkl
../data/rdw-data-2017-04.pkl
../data/rdw-data-2017-05.pkl
../data/rdw-data-2017-06.pkl
../data/rdw-data-2017-07.pkl
../data/rdw-data-2017-08.pkl
../data/rdw-data-2017-09.pkl
../data/rdw-data-2017-10.pkl
../data/rdw-data-2017-11.pkl
../data/rdw-data-2017-12.pkl
../data/rdw-data-2018-01.pkl
../data/rdw-data-2018-02.pkl
../data/rdw-data-2018-03.pkl
../data/rdw-data-2018-04.pkl
../data/rdw-data-2018-05.pkl
../data/rdw-data-2018-06.pkl
../data/rdw-data-2018-07.pkl
../data/rdw-data-2018-08.pkl
../data/rdw-data-2018-09.pkl
../data/rdw-data-2018-10.pkl
../data/rdw-data-2018-11.pkl
../data/rdw-data-2018-12.pkl
../data/rdw-data-2019-01.pkl
../data/rdw-data-2019-02.pkl
../data/rdw-data-2019-03.pkl
../data/rdw-data-2019-04.pkl
../data/rdw-data-2019-05.pkl
../data/rdw-data-2019-06.pkl
../data/rdw-data-2019-07.pkl
../data/rdw-data-2019-08.pkl
../data/rdw-data-2019-09.pkl
../data/rdw-data-2019-10.pkl
../data/rdw-data-2019-11.pkl
../data/rdw-data-2019-12.pkl

las

,Source,Title,Price,Draw,Raw_text,N_images,Images,Note,LotNr,LotType,...,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_14,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_15,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_16,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_17,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_18,rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_19,rdw_versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_1,rdw_versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_1,rdw_versnellingsbak_uitvoering_type_versnellingsbak_1,rdw_TimeStamp_y
2019-12-9058,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900129058,2278.0,False,"[Type ibiza, Kenteken 13-LDD-9, Afgelezen km-s...",9,[http://www.domeinenrz.nl/ufc/static/157492225...,False,K1900129058,Personenauto,...,E,E,F,F,F,NaN,5.0,5.0,H,20191206
2019-12-9059,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900129059,1888.0,False,"[Type passat, Automatic, Kenteken 75-TF-FJ, Af...",9,[http://www.domeinenrz.nl/ufc/static/157492225...,False,K1900129059,Personenauto,...,G,G,G,G,G,NaN,6.0,6.0,A,20191206
2019-12-9061,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900129061,1554.0,False,"[Type leon; 1.6 75 kw bse, Kenteken 58-RX-TX, ...",9,[http://www.domeinenrz.nl/ufc/static/157492225...,False,K1900129061,Personenauto,...,G,G,G,G,G,NaN,5.0,5.0,H,20191206
2019-12-9600,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900129600,755.0,False,"[defecten zijn., Het staat de koper vrij om on...",1,[http://www.domeinenrz.nl/ufc/static/157492225...,False,K1900129600,Partij (ca. 34) Demontage fietsen (Ongesorteer...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-9700,http://www.domeinenrz.nl/catalogi/verkoop_bij_...,Kavel K1900129700,950.0,False,"[Type vh 10, Kenteken OV-64-27, Datum eerste t...",2,[http://www.domeinenrz.nl/ufc/static/157492225...,False,K1900129700,Oplegger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [4]:
# Images

# print first lot
print('Fields with images look like this:\n')
print('{:7s} | {:7s} | {:14s} | {:7s}\n{:7s}-+-{:s}-+-{:s}-+-{:s}'.format('Month','Column','Type','1st row','-'*7,'-'*7,'-'*14,'-'*46))
for k in data:
    col_idx = pd.np.where([c.lower() in ['image','images'] for c in data[k].columns])[0]
    example = data[k].iloc[0,col_idx].values[0]
    print('{:7s} | {:7s} | {}'.format(k,data[k].columns[col_idx][0],type(example)),end='')
    if type(example) == str:
        print('  | "{} .. {}"'.format(example[0:20],example[-20:]))
    else:
        print(' | length {} '.format(len(example)))

Fields with images look like this:

Month   | Column  | Type           | 1st row
--------+---------+----------------+-----------------------------------------------
2017-03 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-04 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-05 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-06 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-07 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-08 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-09 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-10 | Image   | <class 'str'>  | "['http://www.domeine .. 1024/768/image.jpg']"
2017-11 | Images  | <class 'str'>  | "[http://www.domeinen .. /1024/768/image.jpg]"
2017-12 | Images  | <class 'str'>  | "[http://www.domeinen .. /1024/768/image.j

In [5]:
# convert string representation of list to real list
# "['image1', 'image2']"
for k in ['2017-03',
          '2017-04',
          '2017-05',
          '2017-06',
          '2017-07',
          '2017-08',
          '2017-09',
          '2017-10']:
    data[k]['Images'] = data[k]['Image'].apply(eval)
    data[k] = data[k].drop(columns=['Image'])

# convert string representation of list without quotes to real list
# "[image1, image2]"
for k in ['2017-11',
          '2017-12',
          '2018-01',
          '2018-02',
          '2018-03',
          '2018-04']:
    data[k]['Images'] = data[k]['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
    

In [6]:
# Other column name that changed in November 2017
for k in ['2017-03',
          '2017-04',
          '2017-05',
          '2017-06',
          '2017-07',
          '2017-08',
          '2017-09',
          '2017-10']:
    data[k] = data[k].rename(columns={
        'draw':'Draw',
        'nr':'LotNr',
        'raw':'Raw_text',
        'misc':'SupInfo',
        'footnote':'Note',
        'jfq':'jfc',
    })

# Merge all auctions into one data frame

In [7]:
# append data into one data frame
for i,k in enumerate(sorted(data.keys(),reverse=False)):
    if i == 0:
        df = data[k]
    else:
        oldcols = pd.np.setdiff1d(df.columns,data[k].columns)
        newcols = pd.np.setdiff1d(data[k].columns,df.columns)
        if oldcols.size > 0:
            print('fields not present in {:s}'.format(k))
            [print('\t{:s}'.format(c)) for c in oldcols]
        if newcols.size > 0:
            print('fields new in {:s}'.format(k))
            [print('\t{:s}'.format(c)) for c in newcols]
        
        df=df.append(data[k],sort=False).copy()
    print(k,df.shape)
            

2017-03 (234, 381)
fields not present in 2017-04
	rdw_basisgegevens_techn_max_last_koppelpunt_bovengrens
	rdw_motor_uitvoering_brandstof_actieradius_brandstof_extern_oplaadbaar_1_1
	rdw_motor_uitvoering_brandstof_elektrisch_verbruik_combinatie_1_1
	rdw_oplegger_geremd
	rdw_versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_2
	rdw_versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_2
	rdw_versnellingsbak_uitvoering_type_versnellingsbak_2
fields new in 2017-04
	rdw_as_gegevens_gelijkwaardig_aan_luchtvering_1
	rdw_as_gegevens_gelijkwaardig_aan_luchtvering_2
	rdw_basisgegevens_techn_max_massa_autonoom_bovengrens
	rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_13
	rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_14
	rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_15
	rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_16
	rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_17
	rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_18
	rd

# Continue with cars only

In [8]:
# select data
car=df[df.LotType == 'Personenauto'].copy()

In [9]:
# print lots that are cars per auction
prev = ['1999','0','0']
print('Car lots\nYear | Month | First ... Last index')
for ix in car.index:
    y,m,k = ix.split('-')
    if (y != prev[0]) or (m != prev[1]):
        if prev[0] == '1999':
            print('-----+-------+---------------------',end='')
        else:
            print('.. {:s}'.format(prev[2]),end='')
        print('\n{:4s} | {:>5s} |'.format(y,m),end='')
        prev[0] = y
        prev[1] = m
        print(' {:>5s} .'.format(k),end='')
    prev[2] = k
print('.. {:s}'.format(prev[2]),end='')
car.tail()

Car lots
Year | Month | First ... Last index
-----+-------+---------------------
2017 |     3 |  2000 ... 8318
2017 |     4 |  2000 ... 7309
2017 |     5 |  2200 ... 8025
2017 |     6 |  2000 ... 7311
2017 |     7 |  2200 ... 8178
2017 |     8 |  2001 ... 7304
2017 |     9 |  2200 ... 8187
2017 |    10 |  2000 ... 7171
2017 |    11 |  2200 ... 8305
2017 |    12 |  2000 ... 7308
2018 |     1 |  2200 ... 8170
2018 |     2 |  2000 ... 7184
2018 |     3 |  2200 ... 8338
2018 |     4 |  2000 ... 7336
2018 |     5 |  2200 ... 8226
2018 |     6 |  2000 ... 7315
2018 |     7 |  2200 ... 8311
2018 |     8 |  2000 ... 7328
2018 |     9 |  2200 ... 8183
2018 |    10 |  2000 ... 7319
2018 |    11 |  2200 ... 8326
2018 |    12 |  2000 ... 7361
2019 |     1 |  2201 ... 8329
2019 |     2 |  2001 ... 7264
2019 |     3 |  2200 ... 8318
2019 |     4 |  2000 ... 7314
2019 |     5 |  2200 ... 8352
2019 |     6 |  2201 ... 9158
2019 |     7 |  2200 ... 7254
2019 |     8 |  2200 ... 8172
2019 |     9 |  220

,LotNr,Price,Title,LotCat,LotType,ItemBrand,ItemType,Mfdate,Mfyear,APKdate,...,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_86,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_87,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_88,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_89,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_90,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_91,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_92,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_93,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_94,rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_95
2019-12-9056,K1900129056,277.0,Kavel K1900129056,Voertuigen en onderdelen,Personenauto,SKODA,fabia; 50 kw,07.11.2000,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-9057,K1900129057,1401.0,Kavel K1900129057,Voertuigen en onderdelen,Personenauto,OPEL,corsa-c; z1.2xep,22.06.2006,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-9058,K1900129058,2278.0,Kavel K1900129058,Voertuigen en onderdelen,Personenauto,SEAT,ibiza,26.03.2010,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-9059,K1900129059,1888.0,Kavel K1900129059,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,passat,28.07.2006,,08.01.2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-9061,K1900129061,1554.0,Kavel K1900129061,Voertuigen en onderdelen,Personenauto,SEAT,leon; 1.6 75 kw bse,02.11.2005,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save file

In [10]:
# save data
fn = '../data/cars-from-all-auctions.pkl'
# fn = '../../../python-nb/data/cars-from-all-auctions-opbod.pkl'
print(fn)
car.to_pickle(fn)

../data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [11]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [12]:
fn = '../assets/example-table-of-cars.md'
example = car.tail(10).copy()
# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'
table_text = pandas_df_to_markdown_table(example)
with open(fn,'w') as file:
    file.write(table_text)
    
print('A markdown table is available as\n\t{}'.format(fn))

A markdown table is available as
	../assets/example-table-of-cars.md
